- 仿真代价函数：$f_i(x_i,\sigma(x))=100\|x_i-r_i\|^2+\|x_i-\sigma(x)\|^2$

- 新算法：

$$
\begin{aligned}
&y_{i,k+1}=y_{i,k}+\beta\left(\sigma_{i,k}-\sum_{j=1}^Na_{ij}\sigma_{j,k}\right) \\
&x_{i,k+1}=x_{i,k}-\alpha\bigg[\bigtriangledown{}_{1}f_{i}\left(x_{i,k},\sigma_{i,k}\right)+\bigtriangledown\phi_{i}\left(x_{i,k}\right)\left(y_{i,k}+\frac{\bigtriangledown{}_{2}f_{i}\left(x_{i,k},\sigma_{i,k}\right)}{\left[v_{i,k}\right]{}_{i}}\right)\bigg] \\
&v_{i,k+1}=\sum_{j=1}^Na_{ij}v_{j,k} \\
&\sigma_{i,k+1}=\sum_{j=1}^{N}a_{ij}\sigma_{j,k}+\frac{\phi_{i}\left(x_{i,k+1}\right)}{\left[v_{i,k+1}\right]_{i}}-\frac{\phi_{i}\left(x_{i,k}\right)}{\left[v_{i,k}\right]_{i}}
\end{aligned}
$$

- 算法流程：

$$
\begin{aligned}
&y_{i,k+1}=y_{i,k}+\beta\left(\sigma_{i,k+1}-\sum_{j=1}^Na_{ij}\sigma_{j,k+1}\right) \\
&x_{i,k+1}=x_{i,k}-\alpha\bigg[\bigtriangledown{}_{1}f_{i}\left(x_{i,k},\sigma_{i,k}\right)+\left(y_{i,k}+\frac{\bigtriangledown{}_{2}f_{i}\left(x_{i,k},\sigma_{i,k}\right)}{\left[v_{i,k}\right]{}_{i}}\right)\bigg] \\
&v_{i,k+1}=\sum_{j=1}^Na_{ij}v_{j,k} \\
&\sigma_{i,k+1}=\sum_{j=1}^{N}a_{ij}\sigma_{j,k}+\frac{x_{i,k+1}}{\left[v_{i,k+1}\right]_{i}}-\frac{x_{i,k}}{\left[v_{i,k}\right]_{i}} \\
&where \bigtriangledown_1f_i(x_{i, k}, \sigma_{i, k}) = 200(x_i - r_i) + 2(x_i - \sigma_{x_{i, k}}), \bigtriangledown_2f_i(x_{i, k}, \sigma_{i, k}) = -2(x_i - \sigma_{x_{i, k}})
\end{aligned}
$$

- 初始化：$x_{i, 0}$ is arbitrary, $\sigma_{i, 0} = x_{i, 0}$, $v_{i, 0} = e_i$, $y_{i, 0} = 0$, The graph is strongly connected.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Iter_Num = 200
n = 5
x = np.zeros(n)
r = np.zeros(n)
alpha = 0.001
beta = 0.001
v = []
for i in range(n):
    t = []
    for j in range(n):
        if j == i: t.append(1.0)
        else: t.append(0.0)
    v.append(t)
v = np.array(v)
s = x.copy()
y = np.zeros(n)
a = [
    [1/2, 0, 1/2, 0, 0],
    [1/3, 1/3, 0, 0, 1/3],
    [0, 1/2, 1/2, 0, 0],
    [0, 1/2, 0, 1/2, 0],
    [0, 0, 0, 1/2, 1/2]
]
a = np.array(a)
r = np.zeros(n)
ans_x = [[] for i in range(n)] # 存每个点迭代过程中的x坐标  ans_x[i][tim]
ans_y = [[] for i in range(n)] # 存每个点迭代过程中的y坐标  ans_y[i][tim]
s_x = [[] for i in range(n)] # 存每个点迭代过程中关于x坐标的sigma_i  s_x[i][tim]
s_y = [[] for i in range(n)] # 存每个点迭代过程中关于y坐标的sigma_i  s_y[i][tim]
r1 = [3.0, 6.0, 9.0, 6.0, 9.0] # x坐标
r2 = [5.0, 9.0, 8.0, 2.0, 2.0] # y坐标
r1 = np.array(r1)
r2 = np.array(r2)

xx = x.copy()
vv = v.copy()
ss = s.copy()
yy = s.copy()

#-------------------------------------------------------------------

def upd_x(i):
    xx[i] = x[i] - alpha * (200 * (x[i] - r[i]) + 2 * (x[i] - s[i]) + y[i] - 2 * (x[i] - s[i]) / v[i, i])

def upd_v(i):
    sum = np.zeros(n)
    for j in range(n):
        sum += a[i, j] * v[j]
    vv[i] = sum.copy()

def upd_s(i):
    sum = 0
    for j in range(n):
        sum += a[i, j] * s[j]
    ss[i] = sum + xx[i] / vv[i, i] - x[i] / v[i, i]

def upd_y(i):
    sum = 0
    for j in range(n):
        sum += a[i, j] * ss[j]
    yy[i] = y[i] + beta * (ss[i] - sum)

def backup():
    global x, v, s, y
    x = xx.copy()
    v = vv.copy()
    s = ss.copy()
    y = yy.copy()

def init(op):
    global x, r, v, s, y
    global xx, vv, ss, yy
    x = np.random.randint(1, 8, n) # x是(1, 5)的随机数
    x = np.float64(x)
    if (op == 1):
        x[0] = 3.8
        x[1] = 5.2
        x[2] = 3.4
        x[3] = 6.8
        x[4] = 2.6
    else:
        x[0] = 1.8
        x[1] = 6.6
        x[2] = 4.8
        x[3] = 2.0
        x[4] = 5.2
    r = np.zeros(n)
    v = []
    for i in range(n):
        t = []
        for j in range(n):
            if j == i: t.append(1.0)
            else: t.append(0.0)
        v.append(t)
    v = np.array(v)
    s = x.copy()
    y = np.zeros(n)
    xx = x.copy()
    vv = v.copy()
    ss = s.copy()
    yy = s.copy()

def cal(iter_num):
    sum = 0
    for i in range(n):
        sum += 100 * ((ans_x[i][iter_num - 1] - r1[i]) ** 2 + (ans_y[i][iter_num - 1] - r2[i]) ** 2) + \
            (ans_x[i][iter_num - 1] - s_x[i][iter_num - 1]) ** 2 + (ans_y[i][iter_num - 1] - s_y[i][iter_num - 1]) ** 2
    return sum

def solve(iter_num):
    global r, ans_x, ans_y, s_x, s_y
    init(1)
    r[0] = 3.0
    r[1] = 6.0
    r[2] = 9.0
    r[3] = 6.0
    r[4] = 9.0
    for T in range(iter_num):
        for i in range(n):
            upd_x(i)
            ans_x[i].append(xx[i])
            upd_v(i)
            upd_s(i)
            s_x[i].append(ss[i])
            upd_y(i)
        backup()
    init(2)
    r[0] = 5.0
    r[1] = 9.0
    r[2] = 8.0
    r[3] = 2.0
    r[4] = 2.0
    for T in range(iter_num):
        for i in range(n):
            upd_x(i)
            ans_y[i].append(xx[i])
            upd_v(i)
            upd_s(i)
            s_y[i].append(ss[i])
            upd_y(i)
        backup()

solve(Iter_Num)
print('经过%d轮迭代, 最终F(x)收敛到: %.2f' % (Iter_Num, cal(Iter_Num)))
print()

col_lt1 = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']
col_lt2 = ['tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

# Fig 1
print('Fig. 1 (x, y)走势图(x为起点, .为终点)')
plt.figure(figsize=(5,5),dpi=1000)
plt.xlabel('The first coordinate')
plt.ylabel('The second coordinate')
for i in range(n):
    plt.text(r1[i]-0.4, r2[i], 'r'+str(i+1), fontsize=7)
    plt.plot(r1[i], r2[i], marker='+', color=col_lt2[i], markersize='7', markeredgewidth=2)
plt.plot([r1[0], r1[1]], [r2[0], r2[1]], color='silver', linestyle='dotted')
plt.plot([r1[1], r1[2]], [r2[1], r2[2]], color='silver', linestyle='dotted')
plt.plot([r1[2], r1[4]], [r2[2], r2[4]], color='silver', linestyle='dotted')
plt.plot([r1[4], r1[3]], [r2[4], r2[3]], color='silver', linestyle='dotted')
plt.plot([r1[3], r1[0]], [r2[3], r2[0]], color='silver', linestyle='dotted')
for i in range(n):  
    plt.text(ans_x[i][0]-0.4, ans_y[i][0]-0.3, '('+str(round(ans_x[i][0], 2))+','+str(round(ans_y[i][0], 2))+')', fontsize=7)
    plt.text(ans_x[i][0]-0.3, ans_y[i][0]-0.6, 'agent '+str(i+1), fontsize=7)
    plt.plot(ans_x[i][0], ans_y[i][0], marker='x', color=col_lt1[i], markersize='5', markeredgewidth=1.5) # 黑色x为起点
    plt.plot(ans_x[i], ans_y[i], linewidth=0.7, color=col_lt1[i], linestyle='-')
    plt.text(ans_x[i][Iter_Num - 1]-0.35, ans_y[i][Iter_Num - 1]-0.3, '('+str(round(ans_x[i][Iter_Num - 1], 2))+','+str(round(ans_y[i][Iter_Num - 1], 2))+')', fontsize=7)
    plt.plot(ans_x[i][Iter_Num - 1], ans_y[i][Iter_Num - 1], marker = '.', color=col_lt1[i]) # 黑色.为终点
plt.show()

# Fig 2
# print('2. x/y坐标的走势图')
# plt.figure(figsize=(5,5),dpi=1000)
# plt.xlabel('Iteration k')
# plt.ylabel('$x_i^k$')
# for i in range(n):
#     plt.plot(range(1, Iter_Num+1), ans_x[i], linestyle='dashed', linewidth=0.3, color=col_lt1[i])
#     plt.plot(range(1, Iter_Num+1), ans_y[i], linestyle='dashed', linewidth=0.3, color=col_lt1[i])
# plt.show()

# Fig 3
# print(s_x[0][Iter_Num-1])
# print(s_y[0][Iter_Num-1])
# print('3. x/y坐标sigma的走势图')
# plt.figure(figsize=(5,5),dpi=1000)
# plt.xlabel('Iteration k')
# plt.ylabel('$\sigma_i^k$')
# for i in range(n):
#     plt.plot(range(1, Iter_Num + 1), s_x[i], linestyle='dashed', linewidth=0.3, color=col_lt1[i])
#     plt.plot(range(1, Iter_Num + 1), s_y[i], linestyle='dashed', linewidth=0.3, color=col_lt1[i])
# plt.show()

# print('4. F(x)的走势图')
# plt.xlabel('Iter_Num')
# plt.ylabel('F(x)')
# plt.plot(range(1, Iter_Num + 1), [cal(i) for i in range(1, Iter_Num + 1)])
# plt.show()